## Data Quality Control and Cleaning

This section performs quality checks on the raw data and creates cleaned versions with full documentation of all changes made.

# oBDS Data Import and Analysis

This notebook imports oBDS mapping data from Excel files with multiple sheets (modules) and prepares it for quality analysis.

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Display options for better data viewing
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

## File Path Configuration

Specify the path to your oBDS Excel file here.

In [2]:
# TODO: Update this path to point to your oBDS Excel file
excel_file_path = "rawData/oBDS_Module_alle_neu.xlsx"

# Check if file exists
if os.path.exists(excel_file_path):
    print(f"✓ File found: {excel_file_path}")
else:
    print(f"✗ File not found: {excel_file_path}")
    print("Please update the file path above.")

✓ File found: rawData/oBDS_Module_alle_neu.xlsx


## Discover Sheet Names

First, let's see what sheets (modules) are available in the Excel file.

In [3]:
# Read Excel file and get sheet names
try:
    excel_file = pd.ExcelFile(excel_file_path)
    sheet_names = excel_file.sheet_names
    
    print(f"Found {len(sheet_names)} sheets in the Excel file:")
    for i, sheet in enumerate(sheet_names, 1):
        print(f"{i:2d}. {sheet}")
        
except FileNotFoundError:
    print("Excel file not found. Please check the file path.")
    sheet_names = []

Found 19 sheets in the Excel file:
 1. Modul 5
 2. Modul 6
 3. Modul 9
 4. Modul 10
 5. Modul 11
 6. Modul 12
 7. Modul 13
 8. Modul 14
 9. Modul 15
10. Modul 16
11. Modul 17
12. Modul 18
13. Modul 19
14. Modul 20
15. Modul 21
16. Modul 22
17. Modul 23
18. Modul 24
19. Modul 25


## Import All Sheets

Load all sheets into a dictionary of DataFrames.

In [4]:
# Dictionary to store all sheets
obds_data = {}

if sheet_names:
    for sheet_name in sheet_names:
        try:
            df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
            obds_data[sheet_name] = df
            print(f"✓ Loaded sheet '{sheet_name}': {df.shape[0]} rows, {df.shape[1]} columns")
        except Exception as e:
            print(f"✗ Error loading sheet '{sheet_name}': {e}")
    
    print(f"\nTotal sheets loaded: {len(obds_data)}")
else:
    print("No sheets to load.")

✓ Loaded sheet 'Modul 5': 35 rows, 27 columns
✓ Loaded sheet 'Modul 6': 26 rows, 24 columns
✓ Loaded sheet 'Modul 9': 5 rows, 24 columns
✓ Loaded sheet 'Modul 10': 17 rows, 24 columns
✓ Loaded sheet 'Modul 11': 19 rows, 24 columns
✓ Loaded sheet 'Modul 12': 9 rows, 24 columns
✓ Loaded sheet 'Modul 13': 94 rows, 26 columns
✓ Loaded sheet 'Modul 14': 202 rows, 26 columns
✓ Loaded sheet 'Modul 15': 14 rows, 24 columns
✓ Loaded sheet 'Modul 16': 42 rows, 24 columns
✓ Loaded sheet 'Modul 17': 41 rows, 24 columns
✓ Loaded sheet 'Modul 18': 9 rows, 24 columns
✓ Loaded sheet 'Modul 19': 23 rows, 24 columns
✓ Loaded sheet 'Modul 20': 9 rows, 24 columns
✓ Loaded sheet 'Modul 21': 2 rows, 24 columns
✓ Loaded sheet 'Modul 22': 2 rows, 24 columns
✓ Loaded sheet 'Modul 23': 14 rows, 24 columns
✓ Loaded sheet 'Modul 24': 7 rows, 24 columns
✓ Loaded sheet 'Modul 25': 9 rows, 24 columns

Total sheets loaded: 19


## Examine Data Structure

Let's look at the column structure of the first available sheet to understand the data format.

In [5]:
if obds_data:
    # Get the first sheet for examination
    first_sheet_name = list(obds_data.keys())[0]
    first_df = obds_data[first_sheet_name]
    
    print(f"Examining structure of sheet: '{first_sheet_name}'")
    print(f"Shape: {first_df.shape}")
    print("\nColumn names:")
    
    for i, col in enumerate(first_df.columns):
        print(f"{chr(65+i):2s} ({i:2d}): {col}")
    
    print("\nFirst few rows:")
    display(first_df.head())

Examining structure of sheet: 'Modul 5'
Shape: (35, 27)

Column names:
A  ( 0): Identifier
B  ( 1): Feldbezeichnung
C  ( 2): Ausprägungen
D  ( 3): SNOMED CT_Code_Nini 
E  ( 4): SNOMED CT_FSN_Nini
F  ( 5): ISO Score_Nini
G  ( 6): SNOMED CT_Code_Sophie 
H  ( 7): SNOMED CT_FSN_Sophie
I  ( 8): ISO Score_Sophie
J  ( 9): SNOMED CT_Code_Paul 
K  (10): SNOMED CT_FSN_Paul
L  (11): ISO
M  (12): SNOMED CT_Code_Lotte
N  (13): SNOMED CT_FSN_Lotte
O  (14): ISO Score_Lotte
P  (15): Unnamed: 15
Q  (16): SNOMED CT_Code_All
R  (17): SNOMED CT_FSN_all
S  (18): Häufigkeit der Codeverwendung
T  (19): Codes gesamt
U  (20): Unnamed: 20
V  (21): Unnamed: 21
W  (22): SNOMED CT_Code_Konsens
X  (23): SNOMED CT_FSN_Konsens
Y  (24): ISO_Score_Konsens
Z  (25): nan
[  (26): German Edition: Version: 2024-05-15

First few rows:


,Identifier,Feldbezeichnung,Ausprägungen,SNOMED CT_Code_Nini,SNOMED CT_FSN_Nini,ISO Score_Nini,SNOMED CT_Code_Sophie,SNOMED CT_FSN_Sophie,ISO Score_Sophie,SNOMED CT_Code_Paul,SNOMED CT_FSN_Paul,ISO,SNOMED CT_Code_Lotte,SNOMED CT_FSN_Lotte,ISO Score_Lotte,Unnamed: 15,SNOMED CT_Code_All,SNOMED CT_FSN_all,Häufigkeit der Codeverwendung,Codes gesamt,Unnamed: 20,Unnamed: 21,SNOMED CT_Code_Konsens,SNOMED CT_FSN_Konsens,ISO_Score_Konsens,NaN,German Edition: Version: 2024-05-15
0,5.0,Diagnose,NaN,4.394010e+08,Diagnosis (observable entity),0.0,439401001.0,Diagnosis (observable entity),0.0,4.394010e+08,Diagnosis (observable entity),0.0,439401001 (SCT),Diagnosis (observable entity),0.0,NaN,4.394010e+08,Diagnosis (observable entity),4.0,1,1.0,NaN,NaN,NaN,NaN,0.00,439401001 Diagnose
1,5.1,Primärtumor Tumordiagnose ICD Code,C00-C97 Bösartige Neubildung?,NaN,NaN,4.0,372087000.0,Primary malignant neoplasm (disorder),1.0,3.633460e+08,Malignant neoplastic disease (disorder),2.0,363346000,Malignant neoplastic disease (disorder),2.0,Identifier,3.633460e+08,Malignant neoplastic disease (disorder),2.0,2,0.5,NaN,NaN,NaN,NaN,2.25,-
2,5.2,ICD VERSION ?,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,0.0,0,0.0,NaN,NaN,NaN,NaN,4.00,-
3,5.3,Freitext,NaN,9.000000e+17,String (foundation metadata concept),1.0,NaN,NaN,4.0,9.000000e+17,String (foundation metadata concept),1.0,NaN,NaN,4.0,NaN,9.000000e+17,String (foundation metadata concept),2.0,2,0.5,NaN,NaN,NaN,NaN,2.50,-
4,5.4,Primärtumor Topographie ICD-O,NaN,3.714800e+08,Anatomic location of neoplasm (observable entity),2.0,399687005.0,Anatomic location of primary malignant neoplas...,1.0,3.996870e+08,Anatomic location of primary malignant neoplas...,1.0,NaN,NaN,4.0,NaN,3.996870e+08,Anatomic location of primary malignant neoplas...,2.0,2,0.5,NaN,NaN,NaN,NaN,2.00,-


## Column Mapping

Based on the project description, the expected column structure is:
- **A-C**: Original oBDS fields
- **D-E**: Nina (SCTID, FSN, ISO-Score)
- **F**: (Additional Nina column?)
- **G-I**: Sophie (SCTID, FSN, ISO-Score)
- **J-L**: Paul (SCTID, FSN, ISO-Score)
- **M-O**: Lotte (SCTID, FSN, ISO-Score)
- **P**: (Gap or additional column?)
- **Q-R**: Thomas MII Onko (SCTID, FSN, ISO-Score)
- **S**: Code usage frequency
- **T**: Total unique codes
- **U**: Code ratio
- **V**: ISO mismatch annotations
- **W-Z**: Additional mapping data (varies by module)
- **AA**: German translations (some modules)

In [6]:
def analyze_column_structure(df, sheet_name):
    """Analyze and categorize columns based on expected structure."""
    
    print(f"\n=== Column Analysis for {sheet_name} ===")
    
    # Expected column ranges (0-indexed)
    column_ranges = {
        'Original_Fields': (0, 3),    # A-C
        'Nina': (3, 6),               # D-F
        'Sophie': (6, 9),             # G-I
        'Paul': (9, 12),              # J-L
        'Lotte': (12, 15),            # M-O
        'Thomas': (15, 18),           # P-R (adjusted)
        'Metrics': (18, 22),          # S-V
        'Additional': (22, None)      # W+
    }
    
    for category, (start, end) in column_ranges.items():
        if end is None:
            cols = df.columns[start:]
        else:
            cols = df.columns[start:end]
        
        if len(cols) > 0:
            print(f"\n{category} ({len(cols)} columns):")
            for i, col in enumerate(cols):
                col_idx = start + i
                excel_col = chr(65 + col_idx) if col_idx < 26 else f"A{chr(65 + col_idx - 26)}"
                print(f"  {excel_col}: {col}")

# Analyze structure for all sheets
for sheet_name, df in obds_data.items():
    analyze_column_structure(df, sheet_name)


=== Column Analysis for Modul 5 ===

Original_Fields (3 columns):
  A: Identifier
  B: Feldbezeichnung
  C: Ausprägungen

Nina (3 columns):
  D: SNOMED CT_Code_Nini 
  E: SNOMED CT_FSN_Nini
  F: ISO Score_Nini

Sophie (3 columns):
  G: SNOMED CT_Code_Sophie 
  H: SNOMED CT_FSN_Sophie
  I: ISO Score_Sophie

Paul (3 columns):
  J: SNOMED CT_Code_Paul 
  K: SNOMED CT_FSN_Paul
  L: ISO

Lotte (3 columns):
  M: SNOMED CT_Code_Lotte
  N: SNOMED CT_FSN_Lotte
  O: ISO Score_Lotte

Thomas (3 columns):
  P: Unnamed: 15
  Q: SNOMED CT_Code_All
  R: SNOMED CT_FSN_all

Metrics (4 columns):
  S: Häufigkeit der Codeverwendung
  T: Codes gesamt
  U: Unnamed: 20
  V: Unnamed: 21

Additional (5 columns):
  W: SNOMED CT_Code_Konsens
  X: SNOMED CT_FSN_Konsens
  Y: ISO_Score_Konsens
  Z: nan
  AA: German Edition: Version: 2024-05-15

=== Column Analysis for Modul 6 ===

Original_Fields (3 columns):
  A: Identifier
  B: Feldbezeichnung
  C: Ausprägungen

Nina (3 columns):
  D: SNOMED CT_Code_Nini 
  E: SN

## Data Quality Overview

Check for missing values, data types, and basic statistics.

In [7]:
def data_quality_summary(df, sheet_name):
    """Provide a data quality summary for a sheet."""
    
    print(f"\n=== Data Quality Summary: {sheet_name} ===")
    print(f"Shape: {df.shape}")
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024:.1f} KB")
    
    # Missing values
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    
    print("\nColumns with missing values:")
    for col in missing[missing > 0].index:
        print(f"  {col}: {missing[col]} ({missing_pct[col]:.1f}%)")
    
    # Data types
    print("\nData types:")
    print(df.dtypes.value_counts())
    
    return {
        'shape': df.shape,
        'missing_values': missing.sum(),
        'missing_pct': (missing.sum() / (df.shape[0] * df.shape[1])) * 100
    }

# Generate quality summaries
quality_summary = {}
for sheet_name, df in obds_data.items():
    quality_summary[sheet_name] = data_quality_summary(df, sheet_name)


=== Data Quality Summary: Modul 5 ===
Shape: (35, 27)
Memory usage: 31.6 KB

Columns with missing values:
  Identifier: 8 (22.9%)
  Feldbezeichnung: 13 (37.1%)
  Ausprägungen: 21 (60.0%)
  SNOMED CT_Code_Nini : 8 (22.9%)
  SNOMED CT_FSN_Nini: 8 (22.9%)
  ISO Score_Nini: 1 (2.9%)
  SNOMED CT_Code_Sophie : 6 (17.1%)
  SNOMED CT_FSN_Sophie: 6 (17.1%)
  ISO Score_Sophie: 1 (2.9%)
  SNOMED CT_Code_Paul : 4 (11.4%)
  SNOMED CT_FSN_Paul: 4 (11.4%)
  ISO: 1 (2.9%)
  SNOMED CT_Code_Lotte: 6 (17.1%)
  SNOMED CT_FSN_Lotte: 6 (17.1%)
  ISO Score_Lotte: 1 (2.9%)
  Unnamed: 15: 34 (97.1%)
  SNOMED CT_Code_All: 5 (14.3%)
  SNOMED CT_FSN_all: 5 (14.3%)
  Häufigkeit der Codeverwendung: 1 (2.9%)
  Unnamed: 21: 25 (71.4%)
  SNOMED CT_Code_Konsens: 35 (100.0%)
  SNOMED CT_FSN_Konsens: 35 (100.0%)
  ISO_Score_Konsens: 35 (100.0%)
  nan: 1 (2.9%)

Data types:
float64    15
object     12
dtype: int64

=== Data Quality Summary: Modul 6 ===
Shape: (26, 24)
Memory usage: 22.7 KB

Columns with missing values:
 

## Extract Mapper Data

Extract SCTID mappings from each mapper for further analysis.

In [8]:
def extract_mapper_data(df, sheet_name):
    """Extract SCTID mappings from each mapper."""
    
    # This function will need to be adjusted based on actual column names
    # For now, we'll create a template structure
    
    mapper_data = {
        'sheet': sheet_name,
        'mappers': {}
    }
    
    # Expected mapper column positions (will need adjustment)
    mapper_positions = {
        'Nina': {'sctid_col': 3, 'fsn_col': 4, 'iso_col': 5},
        'Sophie': {'sctid_col': 6, 'fsn_col': 7, 'iso_col': 8},
        'Paul': {'sctid_col': 9, 'fsn_col': 10, 'iso_col': 11},
        'Lotte': {'sctid_col': 12, 'fsn_col': 13, 'iso_col': 14},
        'Thomas': {'sctid_col': 15, 'fsn_col': 16, 'iso_col': 17}
    }
    
    for mapper_name, positions in mapper_positions.items():
        try:
            sctid_col = positions['sctid_col']
            if sctid_col < len(df.columns):
                sctids = df.iloc[:, sctid_col]
                mapper_data['mappers'][mapper_name] = {
                    'sctids': sctids.tolist(),
                    'non_null_count': sctids.notna().sum(),
                    'unique_codes': sctids.nunique()
                }
            else:
                print(f"Warning: Column index {sctid_col} for {mapper_name} not found in {sheet_name}")
        except Exception as e:
            print(f"Error extracting data for {mapper_name} in {sheet_name}: {e}")
    
    return mapper_data

# Extract mapper data from all sheets
mapper_extracts = {}
for sheet_name, df in obds_data.items():
    mapper_extracts[sheet_name] = extract_mapper_data(df, sheet_name)
    
print("Mapper data extraction completed.")

Mapper data extraction completed.


## Summary Statistics

Display overall project statistics.

In [9]:
if obds_data:
    print("=== oBDS Mapping Project Summary ===")
    print(f"Total modules (sheets): {len(obds_data)}")
    
    total_rows = sum(df.shape[0] for df in obds_data.values())
    total_concepts = sum(df.shape[0] for df in obds_data.values() if df.shape[0] > 0)
    
    print(f"Total concepts to map: {total_concepts}")
    
    print("\nModule breakdown:")
    for sheet_name, df in obds_data.items():
        print(f"  {sheet_name}: {df.shape[0]} concepts")
    
    # Overall data quality
    total_missing = sum(summary['missing_values'] for summary in quality_summary.values())
    total_cells = sum(summary['shape'][0] * summary['shape'][1] for summary in quality_summary.values())
    overall_missing_pct = (total_missing / total_cells) * 100 if total_cells > 0 else 0
    
    print(f"\nOverall missing data: {total_missing} cells ({overall_missing_pct:.1f}%)")
else:
    print("No data loaded. Please check the file path and try again.")

=== oBDS Mapping Project Summary ===
Total modules (sheets): 19
Total concepts to map: 579

Module breakdown:
  Modul 5: 35 concepts
  Modul 6: 26 concepts
  Modul 9: 5 concepts
  Modul 10: 17 concepts
  Modul 11: 19 concepts
  Modul 12: 9 concepts
  Modul 13: 94 concepts
  Modul 14: 202 concepts
  Modul 15: 14 concepts
  Modul 16: 42 concepts
  Modul 17: 41 concepts
  Modul 18: 9 concepts
  Modul 19: 23 concepts
  Modul 20: 9 concepts
  Modul 21: 2 concepts
  Modul 22: 2 concepts
  Modul 23: 14 concepts
  Modul 24: 7 concepts
  Modul 25: 9 concepts

Overall missing data: 3564 cells (24.4%)


## Next Steps

This notebook provides the foundation for importing oBDS mapping data. Next steps could include:

1. **Column Refinement**: Adjust column mappings based on actual data structure
2. **Data Cleaning**: Handle missing values and invalid SNOMED codes
3. **Export for R Analysis**: Prepare data formats for Krippendorff Alpha calculations
4. **Validation**: Check SNOMED code validity against current terminologies
5. **Quality Metrics**: Calculate agreement statistics and identify problematic mappings

Please provide guidance on the actual column structure to refine the data extraction.